This file uses the run generated by `10_year_scale_run.py` and floweaver
to produce a Sankey diagram that maps appointments with HSI events via facility levels.

Below is the instruction to run the file.

### Install floweaver in Anaconda Prompt (if use Jupyter Notebook) / PyCharm Terminal:

pip install floweaver

pip install ipysankeywidget

jupyter nbextension enable --py --sys-prefix ipysankeywidget

jupyter notebook (to open jupyter notebook, which should be installed first) 

### To display and save the output figures:
Select Start Jupyter Server from the Jupyter Actions Menu (lightbulb icon next to Run All cells icon) -> Open Event Log -> Open in Browser
Or 
Type jupyter notebook in PyCharm Terminal

Find the script and run all cells


In [47]:
import tlo

from pathlib import Path

from tlo.analysis.utils import get_scenario_outputs, load_pickled_dataframes

import pandas as pd

import numpy as np

from matplotlib import pyplot as plt

from ipysankeywidget import SankeyWidget

from floweaver import *

In [48]:
# Declare the name of the file that specified the scenarios used in this run.
scenario_filename = '10_year_scale_run.py'

# Declare usual paths:
# Get the tlo path
tlopath = Path(tlo.__file__).parent.parent.parent
outputspath = tlopath / Path('outputs/bshe@ic.ac.uk')

# Find results folder (most recent run generated using that scenario_filename)
f = -4  # -4: Actual + Default health care seeking, -2: Actual + Maximal health care seeking
results_folder = get_scenario_outputs(scenario_filename, outputspath)[f]
print(f"Results folder is: {results_folder}")

# Declare path for output graphs from this script
make_graph_file_name = lambda stub: results_folder / f"{stub}.png"  # noqa: E731

# Extract results
hsi = pd.read_csv(results_folder / 'hsi_count_by_treatment_appt_level.csv')

# todo: Format data
hsi = hsi[["Appt_Type_Code", "Facility_Level", "Treatment_ID"]].drop_duplicates().reset_index(drop=True)
hsi['Facility_Level'] = 'Facility_Level_' + hsi['Facility_Level'].astype(str)
hsi['source'] = 'Appt_Type_Code'
hsi['target'] = 'Treatment_ID'
hsi['value'] = 1.0

Results folder is: c:\users\jdbb1\pycharmprojects\tlomodel\outputs\bshe@ic.ac.uk\10_year_scale_run-2023-08-01T145207Z_draw0


In [49]:
# Format the flow
size = dict(width=1000, height=1600, margins=dict(left=150, right=520))

# Nodes in alphabetic order
partition_appt_type = Partition.Simple('Appt_Type_Code', pd.array([
    'ConWithDCSA',
    'Under5OPD', 'Over5OPD', 'IPAdmission', 'InpatientDays',
    'AntenatalFirst', 'ANCSubsequent', 'FamPlan', 'EPI', 
    'CompDelivery', 'NormalDelivery', 'Csection',
    'AccidentsandEmerg', 'MajorSurg', 'MinorSurg',
    'U5Malnutr',
    'MentOPD',
    'Mammography', 'DiagRadio', 'Tomography', 'LabMolec', 'LabTBMicro',
    'MaleCirc', 'Peds', 'VCTNegative', 'VCTPositive', 'NewAdult', 'EstNonCom',
    'TBNew', 'TBFollowUp']))
partition_treatment_id = Partition.Simple('Treatment_ID', np.unique(hsi['Treatment_ID']))
partition_facility_level = Partition.Simple('Facility_Level',np.unique(hsi['Facility_Level']))

nodes = {
    'Appt': ProcessGroup(['Appt_Type_Code'], partition_appt_type),
    'HSI': ProcessGroup(['Treatment_ID'], partition_treatment_id),
}

# Add nodes Waypoint
nodes['waypoint'] = Waypoint(partition_facility_level)

bundles = [
    Bundle('Appt', 'HSI', waypoints=['waypoint']),
]

ordering = [
    ['Appt'],  # left
    ['waypoint'],    # middle
    ['HSI'],     # right
    ]


# Set the color for each appt type
palette = {'Under5OPD': 'lightpink', 'Over5OPD': 'lightpink',
           'IPAdmission': 'palevioletred', 'InpatientDays': 'mediumvioletred',
            
           'AntenatalFirst': 'green', 'ANCSubsequent': 'green',
           'FamPlan': 'darkseagreen', 'EPI': 'paleturquoise', 
           'CompDelivery': 'limegreen', 'NormalDelivery': 'limegreen', 'Csection': 'springgreen',
            
           'AccidentsandEmerg': 'darkorange', 'MajorSurg': 'orange', 'MinorSurg': 'gold',
            
           'ConWithDCSA': 'violet',
            
           'U5Malnutr': 'orchid',
            
           'MentOPD': 'darkgrey',
            
           'Mammography': 'lightgrey', 'DiagRadio': 'lightgrey', 'Tomography': 'lightgrey', 
           'LabMolec': 'gainsboro', 'LabTBMicro': 'gainsboro',
            
           'MaleCirc': 'mediumslateblue', 'Peds': 'lightskyblue', 
           'VCTNegative': 'lightsteelblue', 'VCTPositive': 'lightsteelblue', 
           'NewAdult': 'cornflowerblue', 'EstNonCom': 'royalblue',
            
           'TBNew': 'yellow', 'TBFollowUp': 'yellow'}

# Sankey diagram definition (SDD)
sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_appt_type)

# Generate and save Sankey
sankey_appt_level_hsi = weave(sdd, hsi, palette=palette, measures='value').to_widget(**size)

sankey_appt_level_hsi.auto_save_png(make_graph_file_name('Sankey_appt_level_hsi'))

SankeyWidget(groups=[{'id': 'Appt', 'type': 'process', 'title': '', 'nodes': ['Appt^ConWithDCSA', 'Appt^Under5…